In [1]:
from lusidtools.jupyter_tools import toggle_code

"""Households

Demonstration of how to manage the holdings for an investor based on each Mandate & Household they are associated with.

Attributes
----------
instruments
portfolios
properties
set holdings
quotes
portfolio groups
aggregation
"""

toggle_code("Toggle Docstring")

## The Challenge

As a wealth manager you have a number of investors who are associated with multiple investment Mandates. Each Mandate may have one or more Accounts. These investors can be part of one or more Households, and a Household can contain one or more Investors. You would like to be able to see the holdings for an investor based on each Mandate & Household they are associated with. 

You also have a number of Branches and would like to see the assets managed by each Branch. Furthermore each branch has a number of Financial Advisors and you would like to be able to see the assets managed by each Financial Advisor.

## The Solution

1) Create your Instrument universe using a range of identifiers

2) Set up a scope for each branch to hold the accounts

3) Create a LUSID Portfolio for each account

4) Create a Property to hold the investor, mandate, and household details

5) Set the initial holdings of the portfolio

6) Load market data (prices)

7) Create a Portfolio Group for each household

8) Conduct a valuation against each household

9) Consider other ways of grouping & valuing client accounts

In [2]:
# Import LUSID
import lusid
import lusid.models as models
import lusid_sample_data as import_data
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

# Import Libraries
import pprint
from datetime import datetime, timedelta, time
import pytz
import printer as prettyprint
import pandas as pd
import numpy as np
import json
import uuid
import os

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print("LUSID Environment Initialised")
print(
    "LUSID SDK Version: ",
    api_factory.build(lusid.api.ApplicationMetadataApi)
    .get_lusid_versions()
    .build_version,
)

LUSID Environment Initialised
LUSID SDK Version:  0.6.6625.0


![Scopes](img/paper-lusid.gif)

### Create your instrument universe using a range of identifiers

Before you can take on any holdings for your client accounts you need to ensure that your instrument universe has been populated. In this case you will import your instrument universe from a CSV file. Read more about instruments in LUSID in the [LUSID Knowledge Base: Instruments](https://support.lusid.com/what-is-an-instrument).

*Run the cell below to import your instrument universe*

In [3]:
equity_instruments = pd.read_csv("data/households-instruments-equities.csv")
equity_instruments.head(n=20)

,InstrumentName,ClientInternal,Currency,QuotePermId,ExchangeCode,CountryIssue,Ticker,MarketSector,SecurityType,Coupon
0,Amazon_Nasdaq_AMZN,imd_34634534,USD,1-8590928320,UN,united_states_america,AMZN,equity,common_stock,nan
1,Apple_Nasdaq_AAPL,imd_35345345,USD,1-8590932301,UN,united_states_america,AAPL,equity,common_stock,nan
2,VANGUARD LIFESTRATEGY GROWTH,imd_89881022,USD,1-5036190193,US,united_states_america,VASGX,equity,fund_of_funds,nan
3,Salesforce.com Inc,imd_44953022,USD,1-8590926819,US,united_states_america,CRM,equity,common_stock,nan
4,Norwegian Cruise Line Holdings Ltd,imd_44569345,USD,1-21475872704,US,united_states_america,NCLH,equity,common_stock,nan
5,CAMPBELL SOUP CO,imd_44645943,USD,1-8590938673,US,united_states_america,CPB,equity,common_stock,nan
6,Under Armour Inc,imd_44678322,USD,1-8590940860,US,united_states_america,UA,equity,common_stock,nan
7,Halliburton Co,imd_44588822,USD,1-8590930194,US,united_states_america,HAL,equity,common_stock,nan
8,MSCI Inc,imd_44999201,USD,1-8590932969,US,united_states_america,MSCI,equity,common_stock,nan
9,Brown-Forman Corp,imd_45992929,USD,1-8590941544,US,united_states_america,BF/A,equity,common_stock,nan


Now that you have the details for your instruments you can go ahead and create an instrument definition for each instrument. These can then be upserted into LUSID. Read about instrument definitions here [LUSID Knowledge Base: What is an Instrument?](https://support.lusid.com/what-is-an-instrument).

You use an upsert method to add instrument definitions to the instrument universe in LUSID. Read more about the behaviour of the upsert method here [LUSID Knowledge Base: Upsert](https://support.lusid.com/upsert-command).

For further usage of the upsert instruments API call refer to the [LUSID API Docs: Upserting Instruments](https://docs.lusid.com/#operation/UpsertInstruments).

*Run the cell below to upsert your instruments into LUSID*

In [4]:
# Initialise your batch upsert request
batch_upsert_request = {}

# Using your instrument universe create your batch request
for index, instrument in equity_instruments.iterrows():

    # Specify the columns of your identifiers
    identifier_columns = ["QuotePermId", "Ticker", "ClientInternal"]

    # Create your identifiers
    identifiers = {}
    for identifier in identifier_columns:
        identifiers[identifier] = models.InstrumentIdValue(value=instrument[identifier])

    # Build your request and add it to the dictionary
    batch_upsert_request[instrument["InstrumentName"]] = models.InstrumentDefinition(
        name=instrument["InstrumentName"], identifiers=identifiers
    )

# Call LUSID to upsert your instrument definitions
instrument_response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments(
    request_body=batch_upsert_request
)

# Pretty print the response
df = lusid_response_to_data_frame(list(instrument_response.values.values()))
df = df.loc[:, ~df.columns.str.startswith("href")]
df

,lusid_instrument_id,version.effective_from,version.as_at_date,name,identifiers.Ticker,identifiers.ClientInternal,identifiers.Figi,identifiers.LusidInstrumentId,identifiers.QuotePermId,identifiers.Isin,properties,state
0,LUID_PTYEYW95,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Amazon_Nasdaq_AMZN,AMZN,imd_34634534,BBG000BVPXP1,LUID_PTYEYW95,1-8590928320,US0231351067,[],Active
1,LUID_X38YFVJH,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,VANGUARD LIFESTRATEGY GROWTH,VASGX,imd_89881022,NaN,LUID_X38YFVJH,1-5036190193,NaN,[],Active
2,LUID_BP36C9G6,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Apple_Nasdaq_AAPL,AAPL,imd_35345345,BBG000B9XVV8,LUID_BP36C9G6,1-8590932301,US0378331005,[],Active
3,LUID_KY5LY1GS,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,CAMPBELL SOUP CO,CPB,imd_44645943,NaN,LUID_KY5LY1GS,1-8590938673,NaN,[],Active
4,LUID_EBXADDLK,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Salesforce.com Inc,CRM,imd_44953022,NaN,LUID_EBXADDLK,1-8590926819,NaN,[],Active
5,LUID_D32D4U7O,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Brown-Forman Corp,BF/A,imd_45992929,NaN,LUID_D32D4U7O,1-8590941544,NaN,[],Active
6,LUID_2SOUNK42,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,T. Rowe Price Instl Small-Cap Stock Fund,TRSSX,imd_44904222,NaN,LUID_2SOUNK42,1-4295989835,NaN,[],Active
7,LUID_IB1U8CFK,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Under Armour Inc,UA,imd_44678322,NaN,LUID_IB1U8CFK,1-8590940860,NaN,[],Active
8,LUID_KITX6QMD,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Norwegian Cruise Line Holdings Ltd,NCLH,imd_44569345,NaN,LUID_KITX6QMD,1-21475872704,NaN,[],Active
9,LUID_QKZDQQG8,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Blackrock Equity Dividend Fund,MDDVX,imd_49302011,NaN,LUID_QKZDQQG8,1-4297570798,NaN,[],Active


![Scopes](img/households-instrumentmaster.gif)

### Set up a scope to hold the accounts

Your wealth managment company has a number of different branches. Each of these branches will be allocated a Scope in LUSID for its accounts. Read more about scopes in the [LUSID Knowledge Base: Scopes](https://support.lusid.com/what-is-a-scope-in-lusid-and-how-is-it-used).

*Run the cell below to create a name for the scope of two of your branches*

In [5]:
# Get the ids for the scopes
scope_ids = [import_data.create_scope_id() for i in range(0, 3)]

# Set up a scope for each branch
branch_singapore_scope = "branch_{}".format(scope_ids[0])
branch_hongkong_scope = "branch_{}".format(scope_ids[1])
branch_scopes = [branch_singapore_scope, branch_hongkong_scope]

# Set up a scope for reporting
reporting_scope = "reporting_{}".format(scope_ids[2])

# Pretty print the responses
prettyprint.heading("Singapore Branch Scope", branch_singapore_scope)
prettyprint.heading("Hong Kong Branch Scope", branch_hongkong_scope)
prettyprint.heading("Reporting Scope", reporting_scope)

Singapore Branch Scope: branch_396e-1e81-c6e6-ea
Hong Kong Branch Scope: branch_396e-1e81-c6e7-3a
Reporting Scope: reporting_396e-1e81-c6e7-4e


![Scopes](img/households-branch.gif)

### Create a portfolio for each account that exists with the branch

Now that you have decided on the name for your scope, you can create the portfolios to represent the client accounts inside this scope. You will import the account details from a CSV file.

*Run the cell below to import your client's account details*

In [6]:
# Import the account details
accounts = pd.read_csv("data/households-accounts.csv")
accounts.head(n=10)

,account_code,account_name,currency,description,primary_acount_owner_id,household_id,other_account_owner_id,financial_advisor_id,mandate_description,mandate_id,branch_id
0,040004-929987648,OneIntlBrokerage,USD,Primary trading account,invstr_7325jhv93,hhd_jxgru45055,NaN,ffid_abkjas9932,To maximise returns within a given risk thresh...,mndt_9mdf9822,Singapore_Singapore
1,040004-778939522,OneIntlBrokerage,USD,College fund,invstr_7325jhv93,hhd_jxgru45055,invstr_bbgj93921,ffid_jbogjh7878,To build a college fund for multiple dependants,mndt_99kljdslf0,Singapore_Singapore
2,204507-250099511,VangETF,USD,Passive exchange traded fund,invstr_a1kf37761,hhd_ab3452342,NaN,ffid_jbogjh7878,To hold 80% in low risk passive instruments an...,mndt_xxku9sdf9,Singapore_Singapore
3,774001-899930233,FidelityOnlineUSEquityOptions,USD,Online US equity & option trades,invstr_bbgj93921,hhd_jxgru45055,NaN,ffid_xxye990221,To maximise return with a high risk tolerance ...,mndt_uu99sadf2,HongKong_SAR
4,400250-613060229,ActiveManagedBlckrock,USD,Actively managed high return investment account,invstr_a1kf37761,hhd_ab3452342,NaN,ffid_jbogjh7878,To hold 80% in low risk passive instruments an...,mndt_xxku9sdf9,Singapore_Singapore
5,769231-448679901,MutualFund,USD,Pooled investment vehicle,invstr_7325jhv93,hhd_ab3452342,NaN,ffid_abkjas9932,To maximise returns within a given risk thresh...,mndt_9mdf9822,Singapore_Singapore


With the account details loaded you can now create your client portfolios.

Note that every portfolio can be referenced by a unique code. Read more about portfolios in the [LUSID Knowledge Base: Portfolios](https://support.lusid.com/what-is-2).

For further usage of the create portfolio API call refer to the [LUSID API Docs: Create Portfolio](https://docs.lusid.com/#operation/CreatePortfolio).

Note that when you create the portolios in the cell below you are creating it with a 'created' date of 1052 days ago. This number is rather arbitary, in practice it should be the date the portfolio came into existence regardless of the system you first created it in, read more about the importance of the created date on a portfolio in the [LUSID Knowledge Base: Importance of Portfolio Creation Date](https://support.lusid.com/importance-of-portfolio-creation-date).

*Run the cell below to create your portfolios*

In [7]:
# A mapping between branch codes and scopes in LUSID
branch_mapping = {
    "Singapore_Singapore": branch_singapore_scope,
    "HongKong_SAR": branch_hongkong_scope,
}

responses = []

# Iterate over the client accounts
for index, account in accounts.iterrows():

    # Set the creation date of your client portfolio
    portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=1052)).isoformat()

    # Build your request to create your client portfolio
    request = models.CreateTransactionPortfolioRequest(
        display_name=account["account_name"],
        code=account["account_code"],
        base_currency=account["currency"],
        description=account["description"],
        created=portfolio_creation_date,
        corporate_action_source_id=None,
        accounting_method="AverageCost",
        sub_holding_keys=None,
        properties=None,
    )

    # Call LUSID to create your client portfolio
    responses.append(
        api_factory.build(lusid.api.TransactionPortfoliosApi).create_portfolio(
            scope=branch_mapping[account["branch_id"]],
            create_transaction_portfolio_request=request,
        )
    )

# Pretty print the response
lusid_response_to_data_frame(responses)

,href,id.scope,id.code,type,display_name,description,created,version.effective_from,version.as_at_date,is_derived,...,links.5.relation,links.5.href,links.5.method,links.6.relation,links.6.href,links.6.method,links.7.relation,links.7.href,links.7.description,links.7.method
0,https://inwaves.lusid.com/api/api/portfolios/b...,branch_396e-1e81-c6e6-ea,040004-929987648,Transaction,OneIntlBrokerage,Primary trading account,2018-05-06 15:58:17.811433+00:00,2018-05-06 15:58:17.811433+00:00,2021-03-23 15:58:18.108265+00:00,False,...,HoldingsAdjustments,https://inwaves.lusid.com/api/api/transactionp...,GET,EntitySchema,https://inwaves.lusid.com/api/api/schemas/enti...,GET,RequestLogs,http://inwaves.lusid.com/app/insights/logs/0HM...,A link to the LUSID Insights website showing a...,GET
1,https://inwaves.lusid.com/api/api/portfolios/b...,branch_396e-1e81-c6e6-ea,040004-778939522,Transaction,OneIntlBrokerage,College fund,2018-05-06 15:58:18.252474+00:00,2018-05-06 15:58:18.252474+00:00,2021-03-23 15:58:18.538695+00:00,False,...,HoldingsAdjustments,https://inwaves.lusid.com/api/api/transactionp...,GET,EntitySchema,https://inwaves.lusid.com/api/api/schemas/enti...,GET,RequestLogs,http://inwaves.lusid.com/app/insights/logs/0HM...,A link to the LUSID Insights website showing a...,GET
2,https://inwaves.lusid.com/api/api/portfolios/b...,branch_396e-1e81-c6e6-ea,204507-250099511,Transaction,VangETF,Passive exchange traded fund,2018-05-06 15:58:18.638896+00:00,2018-05-06 15:58:18.638896+00:00,2021-03-23 15:58:18.924982+00:00,False,...,HoldingsAdjustments,https://inwaves.lusid.com/api/api/transactionp...,GET,EntitySchema,https://inwaves.lusid.com/api/api/schemas/enti...,GET,RequestLogs,http://inwaves.lusid.com/app/insights/logs/0HM...,A link to the LUSID Insights website showing a...,GET
3,https://inwaves.lusid.com/api/api/portfolios/b...,branch_396e-1e81-c6e7-3a,774001-899930233,Transaction,FidelityOnlineUSEquityOptions,Online US equity & option trades,2018-05-06 15:58:19.104661+00:00,2018-05-06 15:58:19.104661+00:00,2021-03-23 15:58:19.295024+00:00,False,...,HoldingsAdjustments,https://inwaves.lusid.com/api/api/transactionp...,GET,EntitySchema,https://inwaves.lusid.com/api/api/schemas/enti...,GET,RequestLogs,http://inwaves.lusid.com/app/insights/logs/0HM...,A link to the LUSID Insights website showing a...,GET
4,https://inwaves.lusid.com/api/api/portfolios/b...,branch_396e-1e81-c6e6-ea,400250-613060229,Transaction,ActiveManagedBlckrock,Actively managed high return investment account,2018-05-06 15:58:19.380791+00:00,2018-05-06 15:58:19.380791+00:00,2021-03-23 15:58:19.632097+00:00,False,...,HoldingsAdjustments,https://inwaves.lusid.com/api/api/transactionp...,GET,EntitySchema,https://inwaves.lusid.com/api/api/schemas/enti...,GET,RequestLogs,http://inwaves.lusid.com/app/insights/logs/0HM...,A link to the LUSID Insights website showing a...,GET
5,https://inwaves.lusid.com/api/api/portfolios/b...,branch_396e-1e81-c6e6-ea,769231-448679901,Transaction,MutualFund,Pooled investment vehicle,2018-05-06 15:58:19.781994+00:00,2018-05-06 15:58:19.781994+00:00,2021-03-23 15:58:19.986025+00:00,False,...,HoldingsAdjustments,https://inwaves.lusid.com/api/api/transactionp...,GET,EntitySchema,https://inwaves.lusid.com/api/api/schemas/enti...,GET,RequestLogs,http://inwaves.lusid.com/app/insights/logs/0HM...,A link to the LUSID Insights website showing a...,GET


![Scopes](img/households-portfolios.gif)

### Create a property to hold the investor and the household

To keep track of the investor associated with each portfolio as well as the household you can make use of LUSID's extensible properties. These allow you to define a bespoke schema for your portfolio objects. Read more about properties in the [LUSID Knowledge Base: Properties](https://support.lusid.com/what-is-a-property). 

For further usage of the create property definition API call refer to the [LUSID API Docs: Create Property Definition](https://docs.lusid.com/#operation/CreatePropertyDefinition).

*Run the cell below to create a property to hold household_id, the account owner ids & the financial advisor id*

In [8]:
# The property codes to create
property_codes = [
    "household_id",
    "primary_acount_owner_id",
    "other_account_owner_id",
    "financial_advisor_id",
    "mandate_id",
    "mandate_description",
    "branch_id",
]

responses = []

# Iterate over the property codes
for property_code in property_codes:

    # Create your request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain="Portfolio",
        scope=reporting_scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        data_type_id=models.ResourceId(scope="system", code="string"),
    )

    # Call LUSID to create your new property
    responses.append(
        api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
            create_property_definition_request=request
        )
    )

# Pretty print the response
df = lusid_response_to_data_frame(list(instrument_response.values.values()))
df = df.loc[:, ~df.columns.str.startswith("href")]
df

,lusid_instrument_id,version.effective_from,version.as_at_date,name,identifiers.Ticker,identifiers.ClientInternal,identifiers.Figi,identifiers.LusidInstrumentId,identifiers.QuotePermId,identifiers.Isin,properties,state
0,LUID_PTYEYW95,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Amazon_Nasdaq_AMZN,AMZN,imd_34634534,BBG000BVPXP1,LUID_PTYEYW95,1-8590928320,US0231351067,[],Active
1,LUID_X38YFVJH,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,VANGUARD LIFESTRATEGY GROWTH,VASGX,imd_89881022,NaN,LUID_X38YFVJH,1-5036190193,NaN,[],Active
2,LUID_BP36C9G6,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Apple_Nasdaq_AAPL,AAPL,imd_35345345,BBG000B9XVV8,LUID_BP36C9G6,1-8590932301,US0378331005,[],Active
3,LUID_KY5LY1GS,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,CAMPBELL SOUP CO,CPB,imd_44645943,NaN,LUID_KY5LY1GS,1-8590938673,NaN,[],Active
4,LUID_EBXADDLK,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Salesforce.com Inc,CRM,imd_44953022,NaN,LUID_EBXADDLK,1-8590926819,NaN,[],Active
5,LUID_D32D4U7O,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Brown-Forman Corp,BF/A,imd_45992929,NaN,LUID_D32D4U7O,1-8590941544,NaN,[],Active
6,LUID_2SOUNK42,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,T. Rowe Price Instl Small-Cap Stock Fund,TRSSX,imd_44904222,NaN,LUID_2SOUNK42,1-4295989835,NaN,[],Active
7,LUID_IB1U8CFK,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Under Armour Inc,UA,imd_44678322,NaN,LUID_IB1U8CFK,1-8590940860,NaN,[],Active
8,LUID_KITX6QMD,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Norwegian Cruise Line Holdings Ltd,NCLH,imd_44569345,NaN,LUID_KITX6QMD,1-21475872704,NaN,[],Active
9,LUID_QKZDQQG8,0001-01-01 00:00:00+00:00,2021-03-23 15:58:17.317851+00:00,Blackrock Equity Dividend Fund,MDDVX,imd_49302011,NaN,LUID_QKZDQQG8,1-4297570798,NaN,[],Active


You can also create a property which can hold multiple values. These are known as "Collection" properties.

*Run the cell below to create a collection property to hold all account owners*

In [9]:
# The collection property codes to create
property_codes_multi = {
    "account_owners": ["primary_acount_owner_id", "other_account_owner_id"]
}

responses = []

# Iterate over the property codes
for property_code in property_codes_multi.keys():

    # Create your request to define a new property
    request = models.CreatePropertyDefinitionRequest(
        domain="Portfolio",
        scope=reporting_scope,
        code=property_code,
        value_required=False,
        display_name=property_code,
        constraint_style="Collection",  # Note the constraint_style is set to "Collection"
        data_type_id=models.ResourceId(scope="system", code="string"),
    )

    # Call LUSID to create your new property
    responses.append(
        api_factory.build(lusid.api.PropertyDefinitionsApi).create_property_definition(
            create_property_definition_request=request
        )
    )

# Pretty print the response
lusid_response_to_data_frame(responses)

,key,value_type,display_name,data_type_id.scope,data_type_id.code,type,unit_schema,domain,scope,code,...,constraint_style,property_definition_type,property_description,links.0.relation,links.0.href,links.0.method,links.1.relation,links.1.href,links.1.description,links.1.method
0,Portfolio/reporting_396e-1e81-c6e7-4e/account_...,String,account_owners,system,string,Label,NoUnits,Portfolio,reporting_396e-1e81-c6e7-4e,account_owners,...,Collection,ValueProperty,,EntitySchema,https://inwaves.lusid.com/api/api/schemas/enti...,GET,RequestLogs,http://inwaves.lusid.com/app/insights/logs/0HM...,A link to the LUSID Insights website showing a...,GET


With these properties created you can now populate them for each portfolio.

*Run the cell below to populate the values for these properties for each portfolio*

In [10]:
# Iterate over each account
for index, account in accounts.iterrows():

    # Add the relevant "Collection" account properties to the portfolio
    response = api_factory.build(lusid.api.PortfoliosApi).upsert_portfolio_properties(
        scope=branch_mapping[account["branch_id"]],
        code=account["account_code"],
        request_body={
            "Portfolio/{}/{}".format(
                reporting_scope, account_property
            ): models.ModelProperty(
                key="Portfolio/{}/{}".format(reporting_scope, account_property),
                value=models.PropertyValue(
                    label_value_set=models.LabelValueSet(
                        values=[
                            account[field]
                            for field in fields
                            if not pd.isnull(account[field])
                        ]
                    )
                ),
            )
            for account_property, fields in property_codes_multi.items()
        },
    )

    # Pretty print the response
    prettyprint.portfolio_properties_response(response)

    # Add the relevant Single Value account properties to the portfolio
    response = api_factory.build(lusid.api.PortfoliosApi).upsert_portfolio_properties(
        scope=branch_mapping[account["branch_id"]],
        code=account["account_code"],
        request_body={
            "Portfolio/{}/{}".format(
                reporting_scope, account_property
            ): models.ModelProperty(
                key="Portfolio/{}/{}".format(reporting_scope, account_property),
                value=models.PropertyValue(label_value=account[account_property]),
            )
            for account_property in property_codes
            if not pd.isnull(account[account_property])
        },
    )

    # Pretty print the response
    prettyprint.portfolio_properties_response(response)
    print("\n")

Properties Sucessfully Updated for Portfolio
Property key: Portfolio/reporting_396e-1e81-c6e7-4e/account_owners
Values: ['invstr_7325jhv93']

Properties Sucessfully Updated for Portfolio
Property key: Portfolio/reporting_396e-1e81-c6e7-4e/household_id
Value: hhd_jxgru45055

Property key: Portfolio/reporting_396e-1e81-c6e7-4e/primary_acount_owner_id
Value: invstr_7325jhv93

Property key: Portfolio/reporting_396e-1e81-c6e7-4e/mandate_description
Value: To maximise returns within a given risk threshold and focusing on investments with strong ESG scores

Property key: Portfolio/reporting_396e-1e81-c6e7-4e/branch_id
Value: Singapore_Singapore

Property key: Portfolio/reporting_396e-1e81-c6e7-4e/mandate_id
Value: mndt_9mdf9822

Property key: Portfolio/reporting_396e-1e81-c6e7-4e/financial_advisor_id
Value: ffid_abkjas9932



Properties Sucessfully Updated for Portfolio
Property key: Portfolio/reporting_396e-1e81-c6e7-4e/account_owners
Values: ['invstr_bbgj93921', 'invstr_7325jhv93']

Propert

![Scopes](img/households-properties.gif)

### Set the initial holdings of the portfolio

Now that you have your instrument universe populated and portfolios created you can load your current client holdings into their portfolios. In this case you will import their holdings from a CSV file. 

*Run the cell below to import your current client holdings*

In [11]:
# Import and print the holdings
holdings = pd.read_csv("data/households-holdings.csv")
holdings.head(n=20)

,portfolio_code,instrument_name,quantity,price,currency,QuotePermId,branch_id
0,040004-929987648,USD_Cash,25000,1.00,USD,NaN,Singapore_Singapore
1,040004-929987648,Amazon_Nasdaq_AMZN,31,1538.50,USD,1-8590928320,Singapore_Singapore
2,040004-929987648,Salesforce.com Inc,52,152.49,USD,1-8590926819,Singapore_Singapore
3,040004-929987648,Norwegian Cruise Line Holdings Ltd,80,68.98,USD,1-21475872704,Singapore_Singapore
4,040004-929987648,CAMPBELL SOUP CO,150,44.25,USD,1-8590938673,Singapore_Singapore
5,040004-929987648,Under Armour Inc,300,22.52,USD,1-8590940860,Singapore_Singapore
6,040004-929987648,Apple_Nasdaq_AAPL,60,182.19,USD,1-8590932301,Singapore_Singapore
7,040004-778939522,USD_Cash,15000,1.00,USD,NaN,Singapore_Singapore
8,040004-778939522,Brown-Forman Corp,250,59.86,USD,1-8590941544,Singapore_Singapore
9,040004-778939522,Halliburton Co,39,18.54,USD,1-8590930194,Singapore_Singapore


Now that you have imported your client holdings you can add them to LUSID. You can do this by setting the holdings on a portfolio. Read more about how making an adjustment or setting the holdings on a portfolio affects it here [LUSID Knowledge Base: The effect of holding adjustments](https://support.lusid.com/how-do-holding-adjustments-affect-a-portfolio).

For further usage of the set holdings API call refer to the [LUSID API Docs: Set Holdings](https://docs.lusid.com/#operation/SetHoldings).

*Run the cell below to upsert your holdings into LUSID*

In [12]:
# Make the holdings effective from now
holdings_effective_date = datetime.now(pytz.UTC).isoformat()

# Iterate over the portfolios in the holdings CSV
for portfolio in holdings["portfolio_code"].unique():

    # Initialise a list to hold your adjustments
    holding_adjustments = []

    # Iterate over the holdings in each portfolio
    for index, holding in holdings.loc[
        holdings["portfolio_code"] == portfolio
    ].iterrows():

        # Set your instrument identifiers based on whether or not instrument is cash
        if "Cash" in holding["instrument_name"]:
            identifier_key = "Instrument/default/Currency"
            identifer = holding["instrument_name"].split("_")[0]
        else:
            identifier_key = "Instrument/default/QuotePermId"
            identifer = holding["QuotePermId"]

        # Create your holding adjustment and append it to your list
        holding_adjustments.append(
            models.AdjustHoldingRequest(
                instrument_identifiers={identifier_key: identifer},
                tax_lots=[
                    models.TargetTaxLotRequest(
                        units=holding["quantity"],
                        cost=models.CurrencyAndAmount(
                            amount=holding["quantity"] * holding["price"],
                            currency=holding["currency"],
                        ),
                        portfolio_cost=holding["quantity"] * holding["price"],
                        price=holding["price"],
                    )
                ],
            )
        )

    # Call LUSID to set your initial holdings
    response = api_factory.build(lusid.api.TransactionPortfoliosApi).set_holdings(
        scope=branch_mapping[holding["branch_id"]],
        code=portfolio,
        effective_at=holdings_effective_date,
        adjust_holding_request=holding_adjustments,
    )

    # Pretty print our response from LUSID
    prettyprint.set_holdings_response(
        response, branch_mapping[account["branch_id"]], portfolio
    )

Holdings Successfully Set for Portfolio
Scope: branch_396e-1e81-c6e6-ea
Code: 040004-929987648
Holdings Effective From: 2018-05-06 15:58:17.811433+00:00
Holdings Created On: 2021-03-23 15:58:27.997516+00:00

Holdings Successfully Set for Portfolio
Scope: branch_396e-1e81-c6e6-ea
Code: 040004-778939522
Holdings Effective From: 2018-05-06 15:58:18.252474+00:00
Holdings Created On: 2021-03-23 15:58:28.472161+00:00

Holdings Successfully Set for Portfolio
Scope: branch_396e-1e81-c6e6-ea
Code: 204507-250099511
Holdings Effective From: 2018-05-06 15:58:18.638896+00:00
Holdings Created On: 2021-03-23 15:58:28.969218+00:00

Holdings Successfully Set for Portfolio
Scope: branch_396e-1e81-c6e6-ea
Code: 774001-899930233
Holdings Effective From: 2018-05-06 15:58:19.104661+00:00
Holdings Created On: 2021-03-23 15:58:29.444573+00:00

Holdings Successfully Set for Portfolio
Scope: branch_396e-1e81-c6e6-ea
Code: 400250-613060229
Holdings Effective From: 2018-05-06 15:58:19.380791+00:00
Holdings Create

### Load market data prices

With the Portfolio Groups created to group each household together, you can now aggregate across households.

To aggregate & value a portfolio in LUSID you need to upsert market data quotes against the underlying holdings or specify an analytics library to use. Read more about aggregating and valuing portfolios in the [LUSID Knowledge Base: Aggregations and Valuations](https://support.lusid.com/what-is-a-valuation).

In this case you will upsert market data quotes to the quote store to be used in an aggregation request. You will import these quotes from a CSV file.

*Run the cell below to import the market data prices*

In [13]:
# Import the market data prices
prices = pd.read_csv("data/households-prices.csv")
prices.head(n=50)

,instrument_name,currency,QuotePermId,price_current,ticker,client_internal
0,VANGUARD LIFESTRATEGY GROWTH,USD,1-5036190193,33.2,VASGX,imd_34634534
1,Amazon_Nasdaq_AMZN,USD,1-8590928320,"1,738.50",AMZN,imd_35345345
2,Apple_Nasdaq_AAPL,USD,1-8590932301,182.54,AAPL,imd_89881022
3,Salesforce.com Inc,USD,1-8590926819,158.44,CRM,imd_44953022
4,Norwegian Cruise Line Holdings Ltd,USD,1-21475872704,52.38,NCLH,imd_44569345
5,CAMPBELL SOUP CO,USD,1-8590938673,41.93,CPB,imd_44645943
6,Under Armour Inc,USD,1-8590940860,25.76,UA,imd_44678322
7,Halliburton Co,USD,1-8590930194,21.19,HAL,imd_44588822
8,MSCI Inc,USD,1-8590932969,234.5,MSCI,imd_44999201
9,Brown-Forman Corp,USD,1-8590941544,54.1,BF/A,imd_45992929


Now that you have imported the market data you can add it to the quote store in LUSID. Read more about what a quote is in the [LUSID Knowledge Base: What is a Quote?](https://support.lusid.com/what-is-a-quote).

For further usage of the Upsert Quotes API call refer to the [LUSID API Docs: Upsert Quotes](https://docs.lusid.com/#operation/UpsertQuotes).

*Run the cell below to upsert the market data quotes into LUSID*

In [14]:
# Initialise an empty list to hold the market data quotes
instrument_quotes = {}

# Iterate over each quote
for index, quote in prices.iterrows():

    # Get the LUSID Instrument ID for the quoted instrument
    luid = (
        api_factory.build(lusid.api.SearchApi)
        .instruments_search(
            instrument_search_property=[
                models.InstrumentSearchProperty(
                    key="Instrument/default/QuotePermId", value=quote["QuotePermId"]
                )
            ],
            mastered_only=True,
        )[0]
        .mastered_instruments[0]
        .identifiers["LusidInstrumentId"]
        .value
    )

    # Create a quote for this instrument and append it to the list of quotes
    instrument_quotes[luid] = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider="DataScope",
                instrument_id=luid,
                instrument_id_type="LusidInstrumentId",
                quote_type="Price",
                field="Mid",
            ),
            effective_at=holdings_effective_date,
        ),
        metric_value=models.MetricValue(
            value=quote["price_current"], unit=quote["currency"]
        ),
        lineage="InternalSystem",
    )

# Upsert the quotes into LUSID
response = api_factory.build(lusid.api.QuotesApi).upsert_quotes(
    scope=reporting_scope, request_body=instrument_quotes
)

# Pretty print the response
lusid_response_to_data_frame(list(response.values.values()))

,quote_id.quote_series_id.provider,quote_id.quote_series_id.instrument_id,quote_id.quote_series_id.instrument_id_type,quote_id.quote_series_id.quote_type,quote_id.quote_series_id.field,quote_id.effective_at,metric_value.value,metric_value.unit,lineage,cut_label,uploaded_by,as_at
0,DataScope,LUID_X38YFVJH,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,33.20,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00
1,DataScope,LUID_PTYEYW95,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,1738.50,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00
2,DataScope,LUID_BP36C9G6,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,182.54,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00
3,DataScope,LUID_EBXADDLK,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,158.44,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00
4,DataScope,LUID_KITX6QMD,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,52.38,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00
5,DataScope,LUID_KY5LY1GS,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,41.93,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00
6,DataScope,LUID_IB1U8CFK,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,25.76,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00
7,DataScope,LUID_VQBKHWNC,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,21.19,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00
8,DataScope,LUID_MK3KVSBT,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,234.50,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00
9,DataScope,LUID_D32D4U7O,LusidInstrumentId,Price,Mid,2021-03-23T15:58:27.6360720+00:00,54.10,USD,InternalSystem,,00u97x74rdncdc85w2p7,2021-03-23 15:58:32.144444+00:00


### Create a portfolio group for & conduct a valuation against each household

Now that you've created Portfolios for the client accounts, you can group them together into households using Portfolio Groups.

Read more about portfolio groups here [LUSID Knowledge Base: How do you Group and Aggregate Portfolios?](https://support.lusid.com/how-do-you-group-and-aggregate-portfolios)

*Run the cell below to create a function which allows you to create the portfolio groups and add the relevant portfolios*

In [15]:
def create_portfolio_groups(account_data, grouping_key, group_scope, portfolio_scopes):
    """
    param: account_data (DataFrame) - The Pandas DataFrame with the account data
    param: grouping_key (str) - The key to group the accounts by
    param: group_scope (str) - The scope to create the portfolio group in
    param: portfolio_scopes (list[str]) - The list of scopes that contain the
    accounts to group

    returns: N/A
    """

    group_creation_date = (datetime.now(pytz.UTC) - timedelta(days=5000)).isoformat()
    portfolio_creation_date = (datetime.now(pytz.UTC) - timedelta(days=1052)).isoformat()

    # Raise an error saying that the key does not exist in the data
    if grouping_key not in account_data.columns:

        raise (
            """
            The grouping key does not exist in the account data! Please
            check your spelling and that the key exists in the data and 
            try again
            """
        )

    # Get all the unique groups from the grouping key
    for group in account_data[grouping_key].unique():

        # Build a create group request for this group
        group_request = models.CreatePortfolioGroupRequest(
            code=group + "-Group",
            display_name="Contains all accounts for the {} of {}".format(
                grouping_key, group
            ),
            created=group_creation_date,
        )

        try:

            # Call LUSID to create the portfolio group
            response = api_factory.build(
                lusid.api.PortfolioGroupsApi
            ).create_portfolio_group(
                scope=group_scope, create_portfolio_group_request=group_request
            )

            # Pretty print the response
            prettyprint.portfolio_group_response(response, "created")

        except lusid.ApiException as e:
            if json.loads(e.body)["code"] == 128:
                pass
            else:
                print(json.loads(e.body)["title"])

    # Initialise a list to hold all account portfolios
    portfolios = []

    # Iterate over all the scopes
    for portfolio_scope in portfolio_scopes:

        # Call LUSID to list all the portfolios across all the portfolio scopes
        response = api_factory.build(lusid.api.PortfoliosApi).list_portfolios_for_scope(
            scope=portfolio_scope
        )

        # Loop over each portfolio
        for portfolio in response.values:

            # For this portfolio get its properties which you defined earlier
            properties = api_factory.build(
                lusid.api.PortfoliosApi
            ).get_portfolio_properties(scope=portfolio.id.scope, code=portfolio.id.code)

            # Make the list of properties easy to work with by converting them to key-value pairs
            portfolio_properties = properties.properties

            try:

                # Using the group property determine which portfolio group to add this portfolio too
                response = api_factory.build(
                    lusid.api.PortfolioGroupsApi
                ).add_portfolio_to_group(
                    scope=group_scope,
                    code=portfolio_properties[
                        "Portfolio/{}/{}".format(group_scope, grouping_key)
                    ].value.label_value
                    + "-Group",
                    resource_id=models.ResourceId(
                        scope=portfolio.id.scope, code=portfolio.id.code
                    ),
                    effective_at=portfolio_creation_date,
                )

                # Pretty print the response
                prettyprint.get_portfolio_group_response(response)

            except lusid.ApiException as e:
                if json.loads(e.body)["code"] == 171:
                    pass
                else:
                    print(json.loads(e.body)["title"])

Now that you've defined your function you can use it to create portfolio groups for each household.

*Run the cell below to group the portfolios by household*

In [16]:
create_portfolio_groups(
    account_data=accounts,
    grouping_key="household_id",
    group_scope=reporting_scope,
    portfolio_scopes=branch_scopes,
)

Portfolio Group Created
Name: Contains all accounts for the household_id of hhd_jxgru45055
Scope: reporting_396e-1e81-c6e7-4e
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the household_id of hhd_ab3452342
Scope: reporting_396e-1e81-c6e7-4e
Code: hhd_ab3452342-Group
Portfolios Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the household_id of hhd_jxgru45055
Scope: reporting_396e-1e81-c6e7-4e
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the household_id of hhd_jxgru45055
Scope: reporting_396e-1e81-c6e7-4e
Code: hhd_jxgru45055-Group
Portfolios Inside Group: 
040004-929987648
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the household_id of hhd_ab3452342
Scope: reporting_396e-1e81-c6e7-4e
Code: hhd_ab3452342-Group
Portfolios Inside Group: 
204507-250099511
S

Using these portfolio groups you can perform an aggregation across each household. The logic for an aggregation is controled by a LUSID recipe. Read more about recipes in the [LUSID Knowledge Base: What is a Recipe and How Are They Used?](https://support.lusid.com/what-is-a-recipe-and-how-are-they-used).

For further usage of the Get Aggregation by Portfolio API call refer to the [LUSID API Docs: Get Aggregation by Portfolio](https://docs.lusid.com/#operation/GetAggregationByPortfolio).

*Run the cell below to create a function to aggregate and value each household's investments*

In [17]:
recipe_code = "market_value"
recipe_scope = "households"

# Create a recipe to perform a valuation
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)

configuration_recipe = models.ConfigurationRecipe(
    scope=recipe_scope,
    code=recipe_code,
    market=models.MarketContext(
        market_rules=[
            models.MarketDataKeyRule(
                key="Equity.LusidInstrumentId.*",
                supplier="DataScope",
                data_scope=reporting_scope,
                quote_type="Price",
                field="Mid",
            )
        ],
        suppliers=models.MarketContextSuppliers(
            commodity="DataScope",
            credit="DataScope",
            equity="DataScope",
            fx="DataScope",
            rates="DataScope",
        ),
        options=models.MarketOptions(
            default_supplier="DataScope",
            default_instrument_code_type="LusidInstrumentId",
            default_scope=reporting_scope,
        ),
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=models.UpsertRecipeRequest(
            configuration_recipe=configuration_recipe
        )
    )
)

In [18]:
recipe = models.ResourceId(scope=recipe_scope, code=recipe_code)


def aggregate_portfolio_group(grouping_key, group_scope, recipe=recipe):
    """
    param: grouping_key (str) - The key to group the accounts by
    param: group_scope (str) - The scope to the portfolio group is in

    returns: aggregation_results (list[DataFrame]) - The list of resulting
    aggregation dataframes
    """

    # Initialse a list to hold the aggregation results
    aggregation_results_df = []

    # Iterate over each household
    for group in accounts[grouping_key].unique():

        # Create the valuation request
        valuation_request = models.ValuationRequest(
            recipe_id=recipe,
            metrics=[
                models.AggregateSpec(
                    key="Instrument/default/LusidInstrumentId", op="Value"
                ),
                models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
                models.AggregateSpec(key="Holding/default/PV", op="Sum"),
                models.AggregateSpec(key="Holding/default/Units", op="Sum"),
                models.AggregateSpec(key="Instrument/default/Name", op="Value"),
            ],
            group_by=["Instrument/default/Name"],
            portfolio_entity_ids=[
                models.PortfolioEntityId(
                    scope=group_scope,
                    code=group + "-Group",
                    portfolio_entity_type="GroupPortfolio",
                )
            ],
            valuation_schedule=models.ValuationSchedule(
                effective_at=holdings_effective_date
            ),
        )

        # Perform a valuation
        valuation = api_factory.build(lusid.api.AggregationApi).get_valuation(
            valuation_request=valuation_request
        )

        # Pretty print the response
        aggregation_results_df.append(
            prettyprint.aggregation_response_generic_df(
                response=valuation, index_key="Instrument/default/Name", name=group
            )
        )

    return aggregation_results_df

Now that you've defined your function you can use it to aggregate across each household.

*Run the cell below to aggregate across each household*

In [19]:
aggregation_results_df = aggregate_portfolio_group(
    grouping_key="household_id", group_scope=reporting_scope
)

In [20]:
prettyprint.heading("Household", aggregation_results_df[0].columns.name)
aggregation_results_df[0]

Household: hhd_jxgru45055


hhd_jxgru45055,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Amazon_Nasdaq_AMZN,LUID_PTYEYW95,"59,596.86","67,801.5",39.0
Apple_Nasdaq_AAPL,LUID_BP36C9G6,"10,931.4","10,952.4",60.0
Brown-Forman Corp,LUID_D32D4U7O,"14,965.0","13,525.0",250.0
CAMPBELL SOUP CO,LUID_KY5LY1GS,"6,637.5","6,289.5",150.0
Halliburton Co,LUID_VQBKHWNC,"5,991.06","7,183.41",339.0
MSCI Inc,LUID_MK3KVSBT,"36,347.5","36,347.5",155.0
Norwegian Cruise Line Holdings Ltd,LUID_KITX6QMD,"5,518.4","4,190.4",80.0
Salesforce.com Inc,LUID_EBXADDLK,"24,008.48","24,082.88",152.0
USD,CCY_USD,"65,987.0","65,987.0","65,987.0"


In [21]:
prettyprint.heading("Household", aggregation_results_df[1].columns.name)
aggregation_results_df[1]

Household: hhd_ab3452342


hhd_ab3452342,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Blackrock Equity Dividend Fund,LUID_QKZDQQG8,"10,815.64","10,836.56",523.0
T. Rowe Price Instl Small-Cap Stock Fund,LUID_2SOUNK42,"13,555.28","13,449.64",556.0
USD,CCY_USD,"6,523.0","6,523.0","6,523.0"
VANGUARD LIFESTRATEGY GROWTH,LUID_X38YFVJH,"106,600.0","107,900.0","3,250.0"
TOTAL,NaN,"137,493.92","138,709.2","10,852.0"


![Scopes](img/households-portfoliogroupshouseholds.gif)

### Group & aggregate by financial advisor

Perhaps you might also like to group your client accounts by financial advisor. Using your functions you defined earlier you can do this as well.

*Run the cell below to create groups for each advisor and perform aggregations*

In [22]:
# Create the portfolio groups for each financial advisor
create_portfolio_groups(
    account_data=accounts,
    grouping_key="financial_advisor_id",
    group_scope=reporting_scope,
    portfolio_scopes=branch_scopes,
)

# Aggregate across the portfolio groups
aggregation_results_df = aggregate_portfolio_group(
    grouping_key="financial_advisor_id", group_scope=reporting_scope
)

Portfolio Group Created
Name: Contains all accounts for the financial_advisor_id of ffid_abkjas9932
Scope: reporting_396e-1e81-c6e7-4e
Code: ffid_abkjas9932-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the financial_advisor_id of ffid_jbogjh7878
Scope: reporting_396e-1e81-c6e7-4e
Code: ffid_jbogjh7878-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the financial_advisor_id of ffid_xxye990221
Scope: reporting_396e-1e81-c6e7-4e
Code: ffid_xxye990221-Group
Portfolios Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the financial_advisor_id of ffid_jbogjh7878
Scope: reporting_396e-1e81-c6e7-4e
Code: ffid_jbogjh7878-Group
Portfolios Inside Group: 
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the financial_advisor_id of ffid_abkjas9932
Scope: reporting_396e-1e81-c6e7-4e
Code: ffid_abkjas9932-Group
Portfolios Inside Group: 
040004-929987648


In [23]:
prettyprint.heading("Financial Advisor", aggregation_results_df[0].columns.name)
aggregation_results_df[0]

Financial Advisor: ffid_abkjas9932


ffid_abkjas9932,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Amazon_Nasdaq_AMZN,LUID_PTYEYW95,"47,693.5","53,893.5",31.0
Apple_Nasdaq_AAPL,LUID_BP36C9G6,"10,931.4","10,952.4",60.0
CAMPBELL SOUP CO,LUID_KY5LY1GS,"6,637.5","6,289.5",150.0
Norwegian Cruise Line Holdings Ltd,LUID_KITX6QMD,"5,518.4","4,190.4",80.0
Salesforce.com Inc,LUID_EBXADDLK,"7,929.48","8,238.88",52.0
T. Rowe Price Instl Small-Cap Stock Fund,LUID_2SOUNK42,"13,555.28","13,449.64",556.0
USD,CCY_USD,"27,200.0","27,200.0","27,200.0"
Under Armour Inc,LUID_IB1U8CFK,"6,756.0","7,728.0",300.0
TOTAL,NaN,"126,221.56","131,942.32","28,429.0"


In [24]:
prettyprint.heading("Financial Advisor", aggregation_results_df[1].columns.name)
aggregation_results_df[1]

Financial Advisor: ffid_jbogjh7878


ffid_jbogjh7878,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Amazon_Nasdaq_AMZN,LUID_PTYEYW95,"11,903.36","13,908.0",8.0
Blackrock Equity Dividend Fund,LUID_QKZDQQG8,"10,815.64","10,836.56",523.0
Brown-Forman Corp,LUID_D32D4U7O,"14,965.0","13,525.0",250.0
Halliburton Co,LUID_VQBKHWNC,723.06,826.41,39.0
Salesforce.com Inc,LUID_EBXADDLK,"16,079.0","15,844.0",100.0
USD,CCY_USD,"19,323.0","19,323.0","19,323.0"
VANGUARD LIFESTRATEGY GROWTH,LUID_X38YFVJH,"106,600.0","107,900.0","3,250.0"
TOTAL,NaN,"180,409.06","182,162.97","23,493.0"


In [25]:
prettyprint.heading("Financial Advisor", aggregation_results_df[2].columns.name)
aggregation_results_df[2]

Financial Advisor: ffid_xxye990221


ffid_xxye990221,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Halliburton Co,LUID_VQBKHWNC,"5,268.0","6,357.0",300.0
MSCI Inc,LUID_MK3KVSBT,"36,347.5","36,347.5",155.0
USD,CCY_USD,"25,987.0","25,987.0","25,987.0"
Under Armour Inc,LUID_IB1U8CFK,"6,897.0","7,728.0",300.0
TOTAL,NaN,"74,499.5","76,419.5","26,742.0"


![Scopes](img/households-portfoliogroupsadvisor.gif)

### Group & aggregate by investment mandate

You can even group and aggregate by investment mandate. 

*Run the cell below to group and aggregate by investment mandate*

In [26]:
# Create the portfolio groups for each mandate
create_portfolio_groups(
    account_data=accounts,
    grouping_key="mandate_id",
    group_scope=reporting_scope,
    portfolio_scopes=branch_scopes,
)

# Aggregate across the portfolio groups
aggregation_results_df = aggregate_portfolio_group(
    grouping_key="mandate_id", group_scope=reporting_scope
)

Portfolio Group Created
Name: Contains all accounts for the mandate_id of mndt_9mdf9822
Scope: reporting_396e-1e81-c6e7-4e
Code: mndt_9mdf9822-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the mandate_id of mndt_99kljdslf0
Scope: reporting_396e-1e81-c6e7-4e
Code: mndt_99kljdslf0-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the mandate_id of mndt_xxku9sdf9
Scope: reporting_396e-1e81-c6e7-4e
Code: mndt_xxku9sdf9-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the mandate_id of mndt_uu99sadf2
Scope: reporting_396e-1e81-c6e7-4e
Code: mndt_uu99sadf2-Group
Portfolios Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the mandate_id of mndt_99kljdslf0
Scope: reporting_396e-1e81-c6e7-4e
Code: mndt_99kljdslf0-Group
Portfolios Inside Group: 
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the mandate_id of

In [27]:
prettyprint.heading("Mandate", aggregation_results_df[0].columns.name)
aggregation_results_df[0]

Mandate: mndt_9mdf9822


mndt_9mdf9822,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Amazon_Nasdaq_AMZN,LUID_PTYEYW95,"47,693.5","53,893.5",31.0
Apple_Nasdaq_AAPL,LUID_BP36C9G6,"10,931.4","10,952.4",60.0
CAMPBELL SOUP CO,LUID_KY5LY1GS,"6,637.5","6,289.5",150.0
Norwegian Cruise Line Holdings Ltd,LUID_KITX6QMD,"5,518.4","4,190.4",80.0
Salesforce.com Inc,LUID_EBXADDLK,"7,929.48","8,238.88",52.0
T. Rowe Price Instl Small-Cap Stock Fund,LUID_2SOUNK42,"13,555.28","13,449.64",556.0
USD,CCY_USD,"27,200.0","27,200.0","27,200.0"
Under Armour Inc,LUID_IB1U8CFK,"6,756.0","7,728.0",300.0
TOTAL,NaN,"126,221.56","131,942.32","28,429.0"


In [28]:
prettyprint.heading("Mandate", aggregation_results_df[1].columns.name)
aggregation_results_df[1]

Mandate: mndt_99kljdslf0


mndt_99kljdslf0,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Amazon_Nasdaq_AMZN,LUID_PTYEYW95,"11,903.36","13,908.0",8.0
Brown-Forman Corp,LUID_D32D4U7O,"14,965.0","13,525.0",250.0
Halliburton Co,LUID_VQBKHWNC,723.06,826.41,39.0
Salesforce.com Inc,LUID_EBXADDLK,"16,079.0","15,844.0",100.0
USD,CCY_USD,"15,000.0","15,000.0","15,000.0"
TOTAL,NaN,"58,670.42","59,103.41","15,397.0"


In [29]:
prettyprint.heading("Mandate", aggregation_results_df[2].columns.name)
aggregation_results_df[2]

Mandate: mndt_xxku9sdf9


mndt_xxku9sdf9,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Blackrock Equity Dividend Fund,LUID_QKZDQQG8,"10,815.64","10,836.56",523.0
USD,CCY_USD,"4,323.0","4,323.0","4,323.0"
VANGUARD LIFESTRATEGY GROWTH,LUID_X38YFVJH,"106,600.0","107,900.0","3,250.0"
TOTAL,NaN,"121,738.64","123,059.56","8,096.0"


In [30]:
prettyprint.heading("Mandate", aggregation_results_df[3].columns.name)
aggregation_results_df[3]

Mandate: mndt_uu99sadf2


mndt_uu99sadf2,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Halliburton Co,LUID_VQBKHWNC,"5,268.0","6,357.0",300.0
MSCI Inc,LUID_MK3KVSBT,"36,347.5","36,347.5",155.0
USD,CCY_USD,"25,987.0","25,987.0","25,987.0"
Under Armour Inc,LUID_IB1U8CFK,"6,897.0","7,728.0",300.0
TOTAL,NaN,"74,499.5","76,419.5","26,742.0"


![Scopes](img/households-portfoliogroupsmandate.gif)

### Group & aggregate by branch

You can also create a portfolio group to see the investment across all the client accounts across each branch.

*Run the cell below to create a portfolio group for each branch*

In [31]:
# Create the portfolio groups for each mandate
create_portfolio_groups(
    account_data=accounts,
    grouping_key="branch_id",
    group_scope=reporting_scope,
    portfolio_scopes=branch_scopes,
)

# Aggregate across the portfolio groups
aggregation_results_df = aggregate_portfolio_group(
    grouping_key="branch_id", group_scope=reporting_scope
)

Portfolio Group Created
Name: Contains all accounts for the branch_id of Singapore_Singapore
Scope: reporting_396e-1e81-c6e7-4e
Code: Singapore_Singapore-Group
Portfolios Inside Group: 


Portfolio Group Created
Name: Contains all accounts for the branch_id of HongKong_SAR
Scope: reporting_396e-1e81-c6e7-4e
Code: HongKong_SAR-Group
Portfolios Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the branch_id of Singapore_Singapore
Scope: reporting_396e-1e81-c6e7-4e
Code: Singapore_Singapore-Group
Portfolios Inside Group: 
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the branch_id of Singapore_Singapore
Scope: reporting_396e-1e81-c6e7-4e
Code: Singapore_Singapore-Group
Portfolios Inside Group: 
040004-929987648
040004-778939522
Subgroups Inside Group: 


Portfolio Group: 
Name: Contains all accounts for the branch_id of Singapore_Singapore
Scope: reporting_396e-1e81-c6e7-4e
Code: Singapore_Singapore-Group
Portfolios Inside G

In [32]:
prettyprint.heading("Branch", aggregation_results_df[0].columns.name)
aggregation_results_df[0]

Branch: Singapore_Singapore


Singapore_Singapore,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Amazon_Nasdaq_AMZN,LUID_PTYEYW95,"59,596.86","67,801.5",39.0
Apple_Nasdaq_AAPL,LUID_BP36C9G6,"10,931.4","10,952.4",60.0
Blackrock Equity Dividend Fund,LUID_QKZDQQG8,"10,815.64","10,836.56",523.0
Brown-Forman Corp,LUID_D32D4U7O,"14,965.0","13,525.0",250.0
CAMPBELL SOUP CO,LUID_KY5LY1GS,"6,637.5","6,289.5",150.0
Halliburton Co,LUID_VQBKHWNC,723.06,826.41,39.0
Norwegian Cruise Line Holdings Ltd,LUID_KITX6QMD,"5,518.4","4,190.4",80.0
Salesforce.com Inc,LUID_EBXADDLK,"24,008.48","24,082.88",152.0
T. Rowe Price Instl Small-Cap Stock Fund,LUID_2SOUNK42,"13,555.28","13,449.64",556.0


In [33]:
aggregation_results_df

[Singapore_Singapore                      Instrument/default/LusidInstrumentId  \
 Instrument/default/Name                                                         
 Amazon_Nasdaq_AMZN                                              LUID_PTYEYW95   
 Apple_Nasdaq_AAPL                                               LUID_BP36C9G6   
 Blackrock Equity Dividend Fund                                  LUID_QKZDQQG8   
 Brown-Forman Corp                                               LUID_D32D4U7O   
 CAMPBELL SOUP CO                                                LUID_KY5LY1GS   
 Halliburton Co                                                  LUID_VQBKHWNC   
 Norwegian Cruise Line Holdings Ltd                              LUID_KITX6QMD   
 Salesforce.com Inc                                              LUID_EBXADDLK   
 T. Rowe Price Instl Small-Cap Stock Fund                        LUID_2SOUNK42   
 USD                                                                   CCY_USD   
 Under Armour In

In [34]:
prettyprint.heading("Branch", aggregation_results_df[1].columns.name)
aggregation_results_df[1]

Branch: HongKong_SAR


HongKong_SAR,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Halliburton Co,LUID_VQBKHWNC,"5,268.0","6,357.0",300.0
MSCI Inc,LUID_MK3KVSBT,"36,347.5","36,347.5",155.0
USD,CCY_USD,"25,987.0","25,987.0","25,987.0"
Under Armour Inc,LUID_IB1U8CFK,"6,897.0","7,728.0",300.0
TOTAL,NaN,"74,499.5","76,419.5","26,742.0"


![Scopes](img/households-portfoliogroupsscopes.gif)

Finally, you can also create a portfolio group to see the investment across all the client accounts across all branches.

In this case instead of using the functions you defined earlier you will add the group for each branch as sub-groups to the all accounts group.

*Run the cell below to create a portfolio group for all accounts*

In [35]:
# Build a create group request for this group
group_request = models.CreatePortfolioGroupRequest(
    code="allaccounts-Group",
    display_name="Contains all accounts for all branches",
    created=(datetime.now(pytz.UTC) - timedelta(days=5000)).isoformat(),
)

# Call LUSID to create the portfolio group
response = api_factory.build(lusid.api.PortfolioGroupsApi).create_portfolio_group(
    scope=reporting_scope, create_portfolio_group_request=group_request
)

# Pretty print the response
prettyprint.portfolio_group_response(response, "created")

Portfolio Group Created
Name: Contains all accounts for all branches
Scope: reporting_396e-1e81-c6e7-4e
Code: allaccounts-Group
Portfolios Inside Group: 




In [36]:
# Iterate over all the scopes
for branch in accounts["branch_id"].unique():

    # Call LUSID to add the sub-group to the all accounts group
    response = api_factory.build(lusid.api.PortfolioGroupsApi).add_sub_group_to_group(
        scope=reporting_scope,
        code="allaccounts-Group",
        resource_id=models.ResourceId(
            scope=reporting_scope, code="{}-Group".format(branch)
        ),
        effective_at=(datetime.now(pytz.UTC) - timedelta(days=5000)).isoformat(),
    )

    # Pretty print the response
    prettyprint.get_portfolio_group_response(response)

Portfolio Group: 
Name: Contains all accounts for all branches
Scope: reporting_396e-1e81-c6e7-4e
Code: allaccounts-Group
Portfolios Inside Group: 
Subgroups Inside Group: 
Singapore_Singapore-Group


Portfolio Group: 
Name: Contains all accounts for all branches
Scope: reporting_396e-1e81-c6e7-4e
Code: allaccounts-Group
Portfolios Inside Group: 
Subgroups Inside Group: 
Singapore_Singapore-Group
HongKong_SAR-Group




In [37]:
# Create valuation request
valuation_request = models.ValuationRequest(
    recipe_id=recipe,
    metrics=[
        models.AggregateSpec(key="Instrument/default/LusidInstrumentId", op="Value"),
        models.AggregateSpec(key="Holding/default/Cost", op="Sum"),
        models.AggregateSpec(key="Holding/default/PV", op="Sum"),
        models.AggregateSpec(key="Holding/default/Units", op="Sum"),
        models.AggregateSpec(key="Instrument/default/Name", op="Value"),
    ],
    group_by=["Instrument/default/Name"],
    portfolio_entity_ids=[
        models.PortfolioEntityId(
            scope=reporting_scope,
            code="allaccounts-Group",
            portfolio_entity_type="GroupPortfolio",
        )
    ],
    valuation_schedule=models.ValuationSchedule(
        effective_at=holdings_effective_date
    ),
)

# Perform a valuation
valuation_response = api_factory.build(lusid.api.AggregationApi).get_valuation(
    valuation_request=valuation_request
)

aggregation_results_df = prettyprint.aggregation_response_generic_df(
    response=valuation_response,
    index_key="Instrument/default/Name",
    name="All Accounts",
)

In [38]:
prettyprint.heading("All Accounts", "")
aggregation_results_df

All Accounts: 


All Accounts,Instrument/default/LusidInstrumentId,Sum(Holding/default/Cost),Sum(Holding/default/PV),Sum(Holding/default/Units)
Instrument/default/Name,,,,
Amazon_Nasdaq_AMZN,LUID_PTYEYW95,"59,596.86","67,801.5",39.0
Apple_Nasdaq_AAPL,LUID_BP36C9G6,"10,931.4","10,952.4",60.0
Blackrock Equity Dividend Fund,LUID_QKZDQQG8,"10,815.64","10,836.56",523.0
Brown-Forman Corp,LUID_D32D4U7O,"14,965.0","13,525.0",250.0
CAMPBELL SOUP CO,LUID_KY5LY1GS,"6,637.5","6,289.5",150.0
Halliburton Co,LUID_VQBKHWNC,"5,991.06","7,183.41",339.0
MSCI Inc,LUID_MK3KVSBT,"36,347.5","36,347.5",155.0
Norwegian Cruise Line Holdings Ltd,LUID_KITX6QMD,"5,518.4","4,190.4",80.0
Salesforce.com Inc,LUID_EBXADDLK,"24,008.48","24,082.88",152.0


![Scopes](img/households-groupsall.gif)